# GRU for Sentiment Classification

This Gated Recurrent Unit performs sentiment analysis on the IMDB review dataset.

In [ ]:
import keras
from keras.datasets import imdb
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, Dense, Flatten, Dropout, Activation, SpatialDropout1D
from keras.layers import GRU
from keras.callbacks import ModelCheckpoint
import os
from sklearn.metrics import roc_auc_score
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

#### Set Hyperparameters

In [ ]:
output_dir = 'model_output/gru'

epochs = 4
batch_size = 128

n_dim = 64
n_unique_words = 10000 
max_review_length = 100 # lowered due to vanishing gradient over time
pad_type = trunc_type = 'pre'
drop_embed = 0.2 

n_gru = 256 
drop_gru = 0.2

#### Load Data

In [ ]:
(X_train, y_train), (X_valid, y_valid) = imdb.load_data(num_words=n_unique_words)

#### Preprocess Data

In [ ]:
X_train = pad_sequences(X_train, maxlen=max_review_length, padding=pad_type, truncating=trunc_type, value=0)
X_valid = pad_sequences(X_valid, maxlen=max_review_length, padding=pad_type, truncating=trunc_type, value=0)

#### Design Deep Net Architecture

In [ ]:
model = Sequential()
model.add(Embedding(n_unique_words, n_dim, input_length=max_review_length))
model.add(SpatialDropout1D(drop_embed))
model.add(GRU(n_gru, dropout=drop_gru))

model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.summary()

#### Configure the Model

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
modelcheckpoint = ModelCheckpoint(filepath=output_dir+'/weights.{epoch:02d}.hdf5')

In [ ]:
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

### Train the Model

In [ ]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_split=0.3)#, callbacks=[modelcheckpoint])

#### Evaluate

In [ ]:
#model.load_weights(output_dir+'/weights.03.hdf5')
print('Not loading weights.')

In [ ]:
y_hat = model.predict_proba(X_valid)

In [ ]:
y_hat[0]

In [ ]:
plt.hist(y_hat)
_ = plt.axvline(x=0.5, color='orange')

In [ ]:
pct_auc = roc_auc_score(y_valid, y_hat) * 100

In [ ]:
'{:0.2f}'.format(pct_auc)